In [1]:
import xarray as xr
import numpy as np
import json

# Load WRF NetCDF file
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")

# Direct access to variables
u10 = ds["U10"].isel(Time=0)     # shape: (y, x)
v10 = ds["V10"].isel(Time=0)
lat = ds["XLAT"].isel(Time=0)
lon = ds["XLONG"].isel(Time=0)

# Prepare leaflet-velocity JSON format
ny, nx = u10.shape

data = {
    "header": {
        "parameterUnit": "m/s",
        "parameterNumber": 2,
        "parameterNumberName": "eastward_wind",
        "parameterCategory": 2,
        "nx": nx,
        "ny": ny,
        "lo1": float(lon[0, 0]),
        "la1": float(lat[0, 0]),
        "lo2": float(lon[-1, -1]),
        "la2": float(lat[-1, -1]),
        "dx": float((lon[0, -1] - lon[0, 0]) / (nx - 1)),
        "dy": float((lat[0, 0] - lat[-1, 0]) / (ny - 1)),
        "refTime": "2024-02-19 00:00:00",
        "forecastTime": 0
    },
    "uComponent": u10.values.flatten().tolist(),
    "vComponent": v10.values.flatten().tolist()
}

# Save to JSON
with open("wind_velocity.json", "w") as f:
    json.dump(data, f)

print("✅ wind_velocity.json created without wrf-python.")

✅ wind_velocity.json created without wrf-python.


In [6]:
import xarray as xr
import numpy as np
import json

# Load WRF NetCDF file
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")

# Extract U10 and V10 (10 m winds)
u10 = ds["U10"].isel(Time=0)
v10 = ds["V10"].isel(Time=0)
lat = ds["XLAT"].isel(Time=0)
lon = ds["XLONG"].isel(Time=0)

# Convert from m/s to knots
u10_knots = u10.values * 1.94384
v10_knots = v10.values * 1.94384

ny, nx = u10.shape

# Shared header metadata
base_header = {
    "parameterUnit": "KT",
    "parameterCategory": 2,
    "nx": nx,
    "ny": ny,
    "lo1": float(lon[0, 0]),
    "la1": float(lat[0, 0]),
    "lo2": float(lon[-1, -1]),
    "la2": float(lat[-1, -1]),
    "dx": float((lon[0, -1] - lon[0, 0]) / (nx - 1)),
    "dy": float((lat[0, 0] - lat[-1, 0]) / (ny - 1)),
    "refTime": "2024-05-01 00:00:00",
    "forecastTime": 0
}

# Create two objects: eastward and northward winds
eastward = {
    "header": {**base_header, "parameterNumber": 2, "parameterNumberName": "eastward_wind"},
    "data": u10_knots.flatten().tolist()
}

northward = {
    "header": {**base_header, "parameterNumber": 3, "parameterNumberName": "northward_wind"},
    "data": v10_knots.flatten().tolist()
}

# Combine into one array (expected by Leaflet Velocity)
combined = [eastward, northward]

# Write to file
with open("wind_velocity.json", "w") as f:
    json.dump(combined, f)

print("✅ wind_velocity.json created with both eastward and northward wind components.")


✅ wind_velocity.json created with both eastward and northward wind components.


# Prepare Winds JSON for multiple heights

In [20]:
import xarray as xr
import numpy as np
import json
import os

# ---------- CONFIG ----------
INPUT_FILE = "/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc"
OUTPUT_DIR = "/Users/manaruchi/Desktop/WeatherDataViz/raw_data"
DOWNSAMPLE_STEP = 3        # Downsampling grid stride
TIME_STEP_INDEX = 0        # Change this to choose the timestep to export
PRESSURE_LEVELS = [100000, 92500, 85000, 70000, 60000, 50000, 30000, 20000, 10000, 5000]  # Pa
# ----------------------------

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load dataset
ds = xr.open_dataset(INPUT_FILE)

# Read variables
U_stag = ds["U"]
V_stag = ds["V"]
P = ds["P"]
PB = ds["PB"]
PH = ds["PH"]
PHB = ds["PHB"]
XLAT = ds["XLAT"]
XLONG = ds["XLONG"]

# Unstagger U and V for chosen time step
U = 0.5 * (U_stag[TIME_STEP_INDEX, :, :, :-1] + U_stag[TIME_STEP_INDEX, :, :, 1:])
V = 0.5 * (V_stag[TIME_STEP_INDEX, :, :-1, :] + V_stag[TIME_STEP_INDEX, :, 1:, :])

# Total pressure (Pa)
pressure = P + PB
pressure = pressure[TIME_STEP_INDEX]

# Latitude and longitude for chosen time step and downsampled grid
lat = XLAT[TIME_STEP_INDEX]
lon = XLONG[TIME_STEP_INDEX]

lat_ds = lat[::DOWNSAMPLE_STEP, ::DOWNSAMPLE_STEP].values
lon_ds = lon[::DOWNSAMPLE_STEP, ::DOWNSAMPLE_STEP].values

nx = lon_ds.shape[1]
ny = lat_ds.shape[0]

def interpolate_to_pressure(var3d, pressure3d, target_pressure):
    """
    Interpolate var3d (level, y, x) to target pressure (scalar) using linear interp.
    """
    levels, ny, nx = var3d.shape
    var_interp = np.full((ny, nx), np.nan)

    var_rev = var3d[::-1, :, :]
    p_rev = pressure3d[::-1, :, :]

    for j in range(ny):
        for i in range(nx):
            try:
                var_profile = var_rev[:, j, i]
                p_profile = p_rev[:, j, i]
                var_interp[j, i] = np.interp(target_pressure, p_profile, var_profile)
            except Exception:
                var_interp[j, i] = np.nan

    return var_interp




for p_pa in PRESSURE_LEVELS:
    label = f"{int(p_pa/100)}hPa"
    print(f"Processing {label} ...")

    # Interpolate U and V
    u_interp = interpolate_to_pressure(U.values, pressure.values, p_pa)
    v_interp = interpolate_to_pressure(V.values, pressure.values, p_pa)

    # Downsample
    u_ds = u_interp[::DOWNSAMPLE_STEP, ::DOWNSAMPLE_STEP]
    v_ds = v_interp[::DOWNSAMPLE_STEP, ::DOWNSAMPLE_STEP]

    # Build data array [u0, v0, u1, v1, ...]
    data = []
    for i in range(ny):
        for j in range(nx):
            u_val = float(u_ds[i, j])
            v_val = float(v_ds[i, j])
            data.append(u_val)
            data.append(v_val)

    # Leaflet-velocity header
    header = {
        "parameterCategory": 2,
        "parameterNumber": 2,
        "parameterNumberName": "Eastward wind",
        "level": label,
        "nx": nx,
        "ny": ny,
        "dx": float(abs(lon_ds[0,1] - lon_ds[0,0])),
        "dy": float(abs(lat_ds[1,0] - lat_ds[0,0])),
        "la1": float(lat_ds[-1,0]),  # southernmost lat
        "lo1": float(lon_ds[-1,0]),  # westernmost lon
        "la2": float(lat_ds[0,-1]),  # northernmost lat
        "lo2": float(lon_ds[0,-1])   # easternmost lon
    }

    json_obj = {
        "header": header,
        "data": data
    }

    output_file = os.path.join(OUTPUT_DIR, f"wind_{label}.json")
    with open(output_file, "w") as f:
        json.dump(json_obj, f)

    print(f"Saved {output_file}")

print("All done!")


Processing 1000hPa ...
Saved /Users/manaruchi/Desktop/WeatherDataViz/raw_data/wind_1000hPa.json
Processing 925hPa ...
Saved /Users/manaruchi/Desktop/WeatherDataViz/raw_data/wind_925hPa.json
Processing 850hPa ...
Saved /Users/manaruchi/Desktop/WeatherDataViz/raw_data/wind_850hPa.json
Processing 700hPa ...
Saved /Users/manaruchi/Desktop/WeatherDataViz/raw_data/wind_700hPa.json
Processing 600hPa ...
Saved /Users/manaruchi/Desktop/WeatherDataViz/raw_data/wind_600hPa.json
Processing 500hPa ...
Saved /Users/manaruchi/Desktop/WeatherDataViz/raw_data/wind_500hPa.json
Processing 300hPa ...
Saved /Users/manaruchi/Desktop/WeatherDataViz/raw_data/wind_300hPa.json
Processing 200hPa ...
Saved /Users/manaruchi/Desktop/WeatherDataViz/raw_data/wind_200hPa.json
Processing 100hPa ...
Saved /Users/manaruchi/Desktop/WeatherDataViz/raw_data/wind_100hPa.json
Processing 50hPa ...
Saved /Users/manaruchi/Desktop/WeatherDataViz/raw_data/wind_50hPa.json
All done!


In [13]:
u_ds.shape

(11, 66)